In [9]:
'''
Pull price data from Marketstack API
Created JR 09/10/2024
Requires API key from Marketstack. 
'''

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Marketstack')

# Define the securities and date range
symbols = ['AAPL,MSFT']  
end_date = "2024-09-30"  
start_date = "2019-06-30"  

# API URL from marketstack
url = 'http://api.marketstack.com/v1/eod'

# Get results and return dataframe. Created using examples from https://marketstack.com/documentation
def get_eod_data(symbols, start_date, end_date, api_key):
    
    all_data = []  # List to store all records
    limit = 1000 # confirmed max 1000 via docs
    offset = 0

    while True:
        # Define parameters for the API request
        querystring = {
            'access_key': api_key,
            'symbols': symbols,
            'date_from': start_date,
            'date_to': end_date,
            'limit': limit,
            'offset': offset  
        }

        # Make the request
        response = requests.get(url, params=querystring)

        # Check all data has been received
        if response.status_code == 200:
            data = response.json().get('data', [])
            if not data:
                break  

            # Append new data to the list
            all_data.extend(data)

            # Increment the offset to get the next set of data
            offset += limit

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    # Convert the collected data to a pandas DataFrame
    return pd.DataFrame(all_data)

# Fetch all paginated data
df = get_eod_data(symbols, start_date, end_date, API_KEY)

# Show the first few rows of the combined data
print(df.head)

<bound method NDFrame.head of          open      high      low   close      volume    adj_high     adj_low  \
0     428.210  430.4200  425.370  430.30  16807300.0  430.420000  425.370000   
1     230.155  233.0000  229.655  233.00  52779736.0  233.000000  229.650000   
2     228.460  229.5200  227.300  227.79  33993600.0  229.520000  227.300000   
3     431.520  431.8500  427.470  428.02  14888600.0  431.850000  427.470000   
4     435.090  435.3000  429.130  431.31  14454500.0  435.300000  429.130000   
...       ...       ...      ...     ...         ...         ...         ...   
2639  136.800  137.7400  136.295  137.46  13629296.0  136.057128  134.629783   
2640  201.410  203.1323  201.360  202.73  16935217.0  200.743190  198.991734   
2641  136.120  136.5900  135.340  136.58  15237843.0  134.921179  133.686451   
2642  203.170  204.4900  200.650  201.55  27316739.0  202.084921  198.290085   
2643  136.630  136.7000  134.970  135.68  22654160.0  135.029835  133.320971   

       ad

In [10]:
print(df)


df.to_csv('combined_eod_data1.csv', index=False)

         open      high      low   close      volume    adj_high     adj_low  \
0     428.210  430.4200  425.370  430.30  16807300.0  430.420000  425.370000   
1     230.155  233.0000  229.655  233.00  52779736.0  233.000000  229.650000   
2     228.460  229.5200  227.300  227.79  33993600.0  229.520000  227.300000   
3     431.520  431.8500  427.470  428.02  14888600.0  431.850000  427.470000   
4     435.090  435.3000  429.130  431.31  14454500.0  435.300000  429.130000   
...       ...       ...      ...     ...         ...         ...         ...   
2639  136.800  137.7400  136.295  137.46  13629296.0  136.057128  134.629783   
2640  201.410  203.1323  201.360  202.73  16935217.0  200.743190  198.991734   
2641  136.120  136.5900  135.340  136.58  15237843.0  134.921179  133.686451   
2642  203.170  204.4900  200.650  201.55  27316739.0  202.084921  198.290085   
2643  136.630  136.7000  134.970  135.68  22654160.0  135.029835  133.320971   

       adj_close    adj_open  adj_volum

In [9]:
'''
Pull price data from Marketstack API
Created JR 09/10/2024
Requires API key from Marketstack. 
'''

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Marketstack')

# Define the securities and date range
symbols = ['MSFT', 'AAPL']
end_date = "2024-09-30"  
start_date = "2023-09-30"  

# API EOD URL from marketstack
url = 'http://api.marketstack.com/v1/eod'

# Function to fetch EOD data for a single symbol
def get_eod_data(symbol, start_date, end_date, api_key):
    
    all_data = []  
    limit = 1000  # confirmed max 1000 via marketstack docs
    offset = 0

    while True:
        # Define parameters for the API request
        querystring = {
            'access_key': api_key,
            'symbols': symbol,
            'date_from': start_date,
            'date_to': end_date,
            'limit': limit, # required - if missing it only returns the first 1000. 
            'offset': offset  
        }

        # Make the request
        response = requests.get(url, params=querystring)

        # Check all data has been received
        if response.status_code == 200:
            data = response.json().get('data', [])
            if not data:
                break  

            # Append new data to the list
            all_data.extend(data)

            # Increment the offset to get the next set of data. 
            offset += limit

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    # Convert the collected data to a pandas DataFrame
    df = pd.DataFrame(all_data)
    df['symbol'] = symbol  # Add a column for the symbol

    return df

# Initialize an empty DataFrame to store all the data
price_df = pd.DataFrame()

# Loop through each symbol and fetch its data
for symbol in symbols:
    symbol_df = get_eod_data(symbol, start_date, end_date, API_KEY)
    price_df = pd.concat([price_df, symbol_df], ignore_index=True)

print(price_df.head())

# Format the date
price_df = price_df[['date', 'adj_close', 'symbol']]
price_df['date'] = pd.to_datetime(price_df['date']).dt.date

# Switch to array with symbols 
price_df = price_df.pivot(index='date', columns='symbol', values='adj_close')
pivot_df = price_df.reset_index()

print(price_df.head())

# Save to csv to prevent having to make multiple calls whilst testing
price_df.to_csv('combined_eod_data.csv', index=True)


     open    high     low   close      volume  adj_high  adj_low  adj_close  \
0  428.21  430.42  425.37  430.30  16807300.0   430.420   425.37     430.30   
1  431.52  431.85  427.47  428.02  14888600.0   431.850   427.47     428.02   
2  435.09  435.30  429.13  431.31  14454500.0   435.300   429.13     431.31   
3  429.83  433.12  428.57  432.11  13335000.0   433.119   428.57     432.11   
4  433.00  433.35  426.10  429.17  16919500.0   433.350   426.10     429.17   

   adj_open  adj_volume  split_factor  dividend symbol exchange  \
0   428.210  16854606.0           1.0       0.0   MSFT     XNAS   
1   431.520  14896131.0           1.0       0.0   MSFT     XNAS   
2   435.085  14492044.0           1.0       0.0   MSFT     XNAS   
3   429.830  13396364.0           1.0       0.0   MSFT     XNAS   
4   433.000  17015754.0           1.0       0.0   MSFT     XNAS   

                       date  
0  2024-09-30T00:00:00+0000  
1  2024-09-27T00:00:00+0000  
2  2024-09-26T00:00:00+0000  
3 

In [7]:
pivot_df = price_df.pivot(index='date', columns='symbol', values='adj_close')

# Reset the index to make 'date' a column again
pivot_df = pivot_df.reset_index()

# Display the resulting DataFrame
print(pivot_df.head())


symbol        date      AAPL      MSFT
0       2023-10-02  173.3003  320.5554
1       2023-10-03  171.9537  312.1779
2       2023-10-04  173.2105  317.7264
3       2023-10-05  174.4573  318.1248
4       2023-10-06  177.0306  325.9943


In [2]:

price_df = price_df[['date', 'adj_close', 'symbol']]

price_df.to_csv('combined_eod_data.csv', index=False)

NameError: name 'all_symbols_df' is not defined

In [15]:
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    df = get_eod_data(symbol, start_date, end_date, API_KEY)
    
    if not df.empty:
        # Filter to keep only the date and adj_close columns
        df = df[['date', 'adj_close']]
        # Rename the adj_close column to the current symbol
        df = df.rename(columns={'adj_close': symbol})
        # Merge with the main DataFrame, using the date as the key
        if price_df.empty:
            price_df = df
        else:
            price_df = pd.merge(price_df, df, on='date', how='outer')
        print(f"Data for {symbol} collected. {len(df)} records.")
    else:
        print(f"No data available for {symbol}.")

# Sort the DataFrame by date and remove any duplicate dates
price_df = price_df.drop_duplicates(subset=['date']).sort_values('date').reset_index(drop=True)

# Show the first few rows of the combined data
print(price_df.head())

# Save the DataFrame to a CSV file
csv_filename = "all_symbols_eod_data_cleaned.csv"
price_df.to_csv(csv_filename, index=False)

print(f"Cleaned data saved to {csv_filename}.")

Fetching data for MSFT...
Data for MSFT collected. 251 records.
Fetching data for AAPL...
Data for AAPL collected. 251 records.
                       date  adj_close symbol    MSFT_x    AAPL_x    MSFT_y  \
0  2023-10-02T00:00:00+0000   320.5554   MSFT  320.5554  173.3003  320.5554   
1  2023-10-03T00:00:00+0000   312.1779   MSFT  312.1779  171.9537  312.1779   
2  2023-10-04T00:00:00+0000   317.7264   MSFT  317.7264  173.2105  317.7264   
3  2023-10-05T00:00:00+0000   318.1248   MSFT  318.1248  174.4573  318.1248   
4  2023-10-06T00:00:00+0000   325.9943   MSFT  325.9943  177.0306  325.9943   

     AAPL_y  
0  173.3003  
1  171.9537  
2  173.2105  
3  174.4573  
4  177.0306  
Cleaned data saved to all_symbols_eod_data_cleaned.csv.


In [13]:
price_df = price_df[['date', 'adj_close', 'symbol']]

# Convert to a data array (NumPy array) if needed
data_array = price_df.to_numpy()

# Show the first few rows of the filtered DataFrame
print(price_df.head())

# Optional: print the data array to confirm
print(data_array)



                       date  adj_close symbol
0  2024-09-30T00:00:00+0000     430.30   MSFT
1  2024-09-27T00:00:00+0000     428.02   MSFT
2  2024-09-26T00:00:00+0000     431.31   MSFT
3  2024-09-25T00:00:00+0000     432.11   MSFT
4  2024-09-24T00:00:00+0000     429.17   MSFT
[['2024-09-30T00:00:00+0000' 430.3 'MSFT']
 ['2024-09-27T00:00:00+0000' 428.02 'MSFT']
 ['2024-09-26T00:00:00+0000' 431.31 'MSFT']
 ...
 ['2023-10-04T00:00:00+0000' 173.2105 'AAPL']
 ['2023-10-03T00:00:00+0000' 171.9537 'AAPL']
 ['2023-10-02T00:00:00+0000' 173.3003 'AAPL']]


In [12]:
print(price_df)

price_df.to_csv('combined_eod_data_v2.csv', index=False)

       open    high     low   close      volume    adj_high     adj_low  \
0    428.21  430.42  425.37  430.30  16807300.0  430.420000  425.370000   
1    431.52  431.85  427.47  428.02  14888600.0  431.850000  427.470000   
2    435.09  435.30  429.13  431.31  14454500.0  435.300000  429.130000   
3    429.83  433.12  428.57  432.11  13335000.0  433.119000  428.570000   
4    433.00  433.35  426.10  429.17  16919500.0  433.350000  426.100000   
..      ...     ...     ...     ...         ...         ...         ...   
497  173.80  177.99  173.18  177.49  57224100.0  177.535502  172.737784   
498  173.79  175.45  172.68  174.91  48527900.0  175.001988  172.239061   
499  171.09  174.21  170.97  173.66  53020300.0  173.765154  170.533428   
500  172.26  173.63  170.82  172.40  49594600.0  173.186635  170.383811   
501  171.22  174.30  170.93  173.75  52164500.0  173.854924  170.493530   

     adj_close    adj_open  adj_volume  split_factor  dividend symbol  \
0     430.3000  428.210000